# MONKES

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import seaborn as sns
from sklearn.preprocessing import StandardScaler
import pathlib 
import os
import joblib

from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.stattools import adfuller  
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from collections import Counter
from datetime import date 
import datetime


In [ ]:
devices_df = pd.read_csv('devices.csv')
readings_df = pd.read_csv('sampled_readings.csv')
reading_types_df = pd.read_csv('reading_types.csv')

### Outlier Detection Using IQR

In [ ]:
def detect_outliers(df,n,features):
    outlier_indices = []
    
    # iterate over features(columns)
    for col in features:
        # 1st quartile (25%)
        Q1 = np.nanpercentile(df[col], 25)
        # 3rd quartile (75%)
        Q3 = np.nanpercentile(df[col],75)
        # Interquartile range (IQR)
        IQR = Q3 - Q1
        print("First Quartertile:", Q1, ". Third Quartile: ", Q3, ".Interquartile Range: ", IQR)
        # outlier step
        outlier_step = 1.5 * IQR
        
        # Determine a list of indices of outliers for feature col
        outlier_list_col = df[(df[col] < Q1 - outlier_step) | (df[col] > Q3 + outlier_step )].index
        
        # append the found outlier indices for col to the list of outlier indices 
        outlier_indices.extend(outlier_list_col)
        
    outlier_indices = Counter(outlier_indices)        
    multiple_outliers = list( k for k, v in outlier_indices.items() if v >= n )
    
    return multiple_outliers   

In [ ]:
df = readings_df 
for k, v in readings_df.groupby('value_type_id'):
    outliers = detect_outliers(v, 1, ['value'])
    df = df.drop(outliers, axis = 0) 

df.info()

First Quartertile: 428.0 . Third Quartile:  564.8 .Interquartile Range:  136.79999999999995
First Quartertile: 0.0 . Third Quartile:  1.7 .Interquartile Range:  1.7
First Quartertile: 31.0 . Third Quartile:  249.0 .Interquartile Range:  218.0
First Quartertile: 0.0 . Third Quartile:  0.0 .Interquartile Range:  0.0
First Quartertile: 0.0 . Third Quartile:  0.2 .Interquartile Range:  0.2
First Quartertile: 0.0 . Third Quartile:  0.1 .Interquartile Range:  0.1
First Quartertile: 3.8 . Third Quartile:  13.2 .Interquartile Range:  9.399999999999999
First Quartertile: 0.0 . Third Quartile:  31.7 .Interquartile Range:  31.7
First Quartertile: 18.6 . Third Quartile:  20.0 .Interquartile Range:  1.3999999999999986
First Quartertile: 0.0 . Third Quartile:  23.0 .Interquartile Range:  23.0
First Quartertile: 20.4 . Third Quartile:  23.6 .Interquartile Range:  3.200000000000003
First Quartertile: 27.9 . Third Quartile:  45.1 .Interquartile Range:  17.200000000000003
<class 'pandas.core.frame.DataF

### Merging devices with sampled readings
Since devices in the same building_id are situated in the same environment we should expect that they share similar IAQ. There may be differences depending on the # of people in different rooms but we will hypothesize that the difference is minimal. Here we map device_ids to buildings to group all devices by building_id

In [ ]:
#merging devices with sampled readings

df = pd.merge(df, devices_df, on='device_id', how='inner')
df = df.drop('device_id', axis = 1)

### Have aggregate value_types with the same hour
Since the data is not given in consistent time-steps we will use downsampling to aggregate data points for 5 minute time-steps. We will partition the data based on value_type_id as well as building_id

In [ ]:
df['date'] = pd.to_datetime(df['date'])
df['date'] = df['date'].dt.floor('5min') #group datetimes by 5 mins as our timestep 

aggregate_function = {'value': 'mean'}
df = df.groupby(['building_id', 'date', 'value_type_id']).agg(aggregate_function) #aggregate values with same value_type_id, building_id, and timestamp with an average


#pivot table so that value_type_id is a column 
df = pd.pivot_table(df, values = 'value', index = ['date', 'building_id'], columns = 'value_type_id').reset_index()  
df = df.rename_axis(None).rename_axis(None, axis=1)
df.columns = df.columns.map(str)

### Interpolating Small Gaps
For small gaps (15 minutes) in data we will use interpolation to predict missing values 

In [ ]:
interp = pd.DataFrame()
for building, df1 in df.groupby('building_id'):
        df1 = df1.sort_values(by = 'date')
        for i in range (1, 13): 
                cnt = df1[str(i)].count()
                if cnt > 1: 
                        df1[str(i)] = df1[str(i)].interpolate(method='spline', order = min(cnt - 1, 3), limit = 3, axis=0) #spline interpolation of maximum 3 consecutive missing values
                elif cnt == 1: 
                        df1[str(i)] = df1[str(i)].interpolate(method='linear', limit = 3, axis=0) #if there is only 1 value in the value column use linear interpolation
                else:  
                        df1[str(i)] = df1[str(i)].fillna(0) #otherwise fill with 0s 
        interp = pd.concat([interp, df1], ignore_index = True)

In [ ]:
df = interp

In [ ]:
# df.to_csv('preprocessed_lstm.csv') # <-- COMMENT THIS OUT IF YOU DON'T HAVE preprocessd.csv yet 

In [ ]:
print(df.info(verbose = True, show_counts = True))

# Model Creation
Since we have resampled the data into 5 minute segments, the data has been converted into a time series format for each building. Due to the large amount of data we opted to use a LSTM to model trends throughout sequences of 5 minutes. Since we did not want to create multiple models for each value_type_id and building_id due to time constraints, we opted for a multivariate LSTM with one hot encoded building_ids. It should be noted that better results can be achieved through multiple models as correlation between value_type_ids is very minimal according to seaborn's correlation matrix.

## Feature Generation: 
In order to help with model performance we will also generate extra features 
### One hot encoded buildings
We will one hot encode buildings that will be fed into a classifier that is concatenated to the LSTM. This will allow the neural network to adapt to different environments of buildings 
### Work Days and Work Hour
These features will be booleans corresponding to with an hour is a business hour and whether a day is a business day. This is because we noticed decreases in certain values such as temperature while it is not working hours. We suspect this is to save on energy when there is a lack of persons in buildings
### Seasons 
We will also add one hot encoded columns for seasons of Winter, Spring, Summer and Fall. After completing AD fuller tests, we noticed that value types such as Relative Humidity experienced seasonal trends. Adding a seasonal column would help the model recognize these trends. 

In [ ]:
import numpy as np 
import pandas as pd 

from sklearn.model_selection import train_test_split
from sklearn.model_selection import TimeSeriesSplit #for data preprocessing and crass validating 
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error, r2_score


from datetime import date
from datetime import datetime

import tensorflow as tf

import keras
import keras.layers as layers
from keras.layers import Dense
from keras.models import Sequential
from keras.utils import to_categorical
from keras.optimizers import SGD 
from keras.callbacks import EarlyStopping
from keras.losses import MeanSquaredError

import itertools
from keras.layers import LSTM

from scipy.stats import boxcox 
from scipy.special import inv_boxcox

from termcolor import colored


In [ ]:
samples = pd.read_csv('preprocessed_lstm.csv') #reading in necessary datasets

samples['date'] = pd.to_datetime(samples['date']) # one hot encoding buildings
building_encoder = pd.get_dummies(samples['building_id'])
samples = samples.join(building_encoder.add_suffix('_b'))

reading_types = pd.read_csv('reading_types.csv')
samples.info()


df_lst = [(k, v) for k, v in samples.groupby('building_id')] #grouping buildings separately


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1832737 entries, 0 to 1832736
Data columns (total 51 columns):
 #   Column         Dtype         
---  ------         -----         
 0   Unnamed: 0     int64         
 1   date           datetime64[ns]
 2   building_id    int64         
 3   1              float64       
 4   2              float64       
 5   3              float64       
 6   4              float64       
 7   5              float64       
 8   6              float64       
 9   7              float64       
 10  8              float64       
 11  9              float64       
 12  10             float64       
 13  11             float64       
 14  12             float64       
 15  work_hours     bool          
 16  day type       int64         
 17  Fall           bool          
 18  Spring         bool          
 19  Summer         bool          
 20  Winter         bool          
 21  trimester_day  int64         
 22  1_b            bool          
 23  2_b    

Here, buildings are grouped separately when generating predictions and fitting the model. This is because there is the IAQ across buildings can be considered independent of each other and is only dependent on the previous timesteps in the particular building. 

In [ ]:
from matplotlib import pyplot as plt
from sklearn.discriminant_analysis import StandardScaler


value_type_ids = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12']
scaler = StandardScaler()  #standard scaler used to improve performance of LSTM 
scaler = scaler.fit(samples[value_type_ids]) 

trainX, trainY, train_class = np.array([[[]]]).reshape(0, 10, 12), np.array([[]]).reshape(0, 12), np.array([[]]).reshape(0, 35) #create numpy arrays for training


for building, df in df_lst:
    print('building:', building, '-'*80)
    train_dates = pd.to_datetime(df['date'])

    df = df.set_index('date', drop = False) 
    idx = pd.date_range('01-01-2023', '12-31-2023 23:55', freq = '5min')
    df = df.reindex(idx, fill_value = 0) #reindex dataframe to include missing 5min timesteps from training data

    df['date'] = df.index 
    
    df['day type'] = df['date'].dt.dayofweek.map({ #day type 1 for work day 0 for non-work day
        0: 1,
        1: 1,
        2: 1,
        3: 1,
        4: 1,
        5: 0, 
        6: 0
    })

        
    df['season'] = df['date'].dt.month.map({ #creating a mapping for seasons
        1: 'Winter',
        2: 'Winter',
        3: 'Spring',
        4: 'Spring',
        5: 'Spring',
        6: 'Summer',
        7: 'Summer',
        8: 'Summer',
        9: 'Fall',
        10: 'Fall',
        11: 'Fall',
        12: 'Winter'
    })

    df['work_hours'] = df['date'].dt.hour.between(8, 18) #whether an hour is working hour or not
    df['work_hours'].map({True: 1, False: 0})

    df['Winter'] = df['season'].map({ #one hot encoding all seasons 
        'Winter': 1,
        'Spring': 0,
        'Summer': 0,
        'Fall': 0
    })
    df['Spring'] = df['season'].map({
        'Winter': 0,
        'Spring': 1,
        'Summer': 0,
        'Fall': 0
    })
    df['Summer'] = df['season'].map({
        'Winter': 0,
        'Spring': 0,
        'Summer': 1,
        'Fall': 0
    })
    df['Fall'] = df['season'].map({
        'Winter': 0,
        'Spring': 0,
        'Summer': 0,
        'Fall': 1
    })


    df = df.drop('season', axis = 1)

    multivariate = df.drop(['Unnamed: 0', 'building_id', 'date'], axis = 1)  #extracting features to feed into model
    multivariate = multivariate.astype('float32')
    classification_info = multivariate.drop(['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12'], axis = 1) #getting classification info (day type, season, building_ids, working hour)

    df[str(building) + '_b'] = 1 #setting building_id column to 1

    # classification_info.info()

    multivariate = multivariate[['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12']]  #multivariate values to feed into LSTM
    multivariate = multivariate.interpolate(method = 'linear').bfill().ffill() #interpolate unfilled values 

    sz = len(multivariate)
    train_sz = int(sz * 0.9)
    test_sz = len(multivariate) - train_sz

    df_scaled = scaler.transform(multivariate)

    train, test = df_scaled[0:train_sz,:], df_scaled[train_sz:sz,:] #create training and testing (testing is not needed was only for us to observe results)
    tClass, tstClass = classification_info[0: train_sz], classification_info[train_sz:sz]

    test_dates = train_dates[train_sz:sz]

    def create_dataset(dataset, classes, look_back = 1): #generating dataset for lstm 
        dataX, dataY = [], [] #lstm inputs for previous timestep and expected output
        class_X = [] #classification information
        for i in range(len(dataset) - look_back - 1):
            if(np.isnan(dataset[i + look_back]).any()):  #if any value is null do not use current time step
                continue
            dataY.append(dataset[i + look_back, :]) #if no nulls generate inputs for training LSTM
            a = dataset[i:(i + look_back), :] #grabbing last i + look_back timesteps for LSTM
            if(np.isnan(a).any()): # for masking
                a.fill(-1)
            dataX.append(a) 
            class_X.append(classes.iloc[i + look_back, :])

        return np.array(dataX), np.array(dataY), np.array(class_X)
    
    look_back = 10 #look_back value for LSTM 
    tX, tY, tClass = create_dataset(train, tClass, look_back)

    
    # reshape input to be [samples, time steps, features]
    tX, tY, tClass = np.array(tX), np.array(tY), np.array(tClass)

    trainX, trainY, train_class = np.append(trainX, tX, axis = 0), np.append(trainY, tY, axis = 0), np.append(train_class, tClass, axis = 0) #add current building's training data to all training data


print(trainX.shape, trainY.shape, train_class.shape)


In [ ]:
# model creation
value_input = keras.Input(shape = (trainX.shape[1], trainX.shape[2]), name = "values") #LSTM 
lstm1_value = layers.LSTM(150, return_sequences = True)(value_input) # LSTM architecture was generated through trial and error
lstm2_value = layers.LSTM(50, return_sequences = False)(lstm1_value)

classification_input = keras.Input(shape = (train_class.shape[1],), name = "class") #classification neural network

x = layers.concatenate([lstm2_value, classification_input]) #concatenate classification neural network with LSTM 
dense_layer = layers.Dense(30, activation = 'relu')(x) #fully connected layer
val_pred = layers.Dense(12, activation = 'linear', name = "output")(dense_layer) #output layer

model = keras.Model(inputs = [value_input, classification_input], outputs = val_pred)


early_stop = EarlyStopping(monitor = 'val_loss', patience = 5) #early stopping for model to prevent overfitting

model.compile(optimizer= tf.keras.optimizers.Adam(), loss = {"output": tf.keras.losses.Huber() } ) #using a huber loss function to combine properties of Mean Absolute Error and Root Mean Squared Error

history = model.fit({
    "values": trainX, 
    "class": train_class
}, {
    "output": trainY
}, batch_size = 64, validation_split = 0.2, epochs = 50, verbose = 1, callbacks = [early_stop])

Epoch 1/50
19384/34292 [===============>..............] - ETA: 2:06 - loss: 0.0648

In [ ]:
model.save('model2.keras')

# Predicting test data

In [ ]:
import numpy as np 
import pandas as pd 

from datetime import date
from datetime import datetime
from datetime import timedelta

import tensorflow as tf
import keras
import keras.layers as layers
from keras.layers import Dense
from keras.models import Sequential
from keras.utils import to_categorical
from keras.optimizers import SGD 
from keras.callbacks import EarlyStopping
from keras.losses import MeanSquaredError
from keras.layers import LSTM

import itertools

from sklearn.discriminant_analysis import StandardScaler

import joblib
from tqdm.notebook import tqdm 

import swifter

In [ ]:
model = tf.keras.models.load_model('model2.keras') #loading in saved model

In [ ]:
samples = pd.read_csv('preprocessed_lstm.csv')
building_id = pd.read_csv('devices.csv')
test = pd.read_csv('test.csv', header = None)
reading_types = pd.read_csv('reading_types.csv')

samples['date'] = pd.to_datetime(samples['date'])

predictions = test
value_type_ids = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12']

train = pd.DataFrame()

In [ ]:
for building, df in samples.groupby('building_id'):  #generating features for prediction same as for training
    df = df.set_index('date', drop = False) 
    idx = pd.date_range('01-01-2023', '12-31-2023 23:55', freq = '5min')
    df = df.reindex(idx, fill_value = building)
    df.index = pd.to_datetime(df.index)
    df['date'] = df.index

    df = df.drop(['day type', 'Winter', 'Spring', 'Summer', 'Fall', 'work_hours', 'trimester_day'], axis = 1)
    
    df['day type'] = df['date'].dt.dayofweek.map({
        0: 1,
        1: 1,
        2: 1,
        3: 1,
        4: 1,
        5: 0, 
        6: 0
    })

        
    df['season'] = df['date'].dt.month.map({
        1: 'Winter',
        2: 'Winter',
        3: 'Spring',
        4: 'Spring',
        5: 'Spring',
        6: 'Summer',
        7: 'Summer',
        8: 'Summer',
        9: 'Fall',
        10: 'Fall',
        11: 'Fall',
        12: 'Winter'
    })

    df['work_hours'] = df['date'].dt.hour.between(8, 18)
    df['work_hours'].map({True: 1, False: 0})

    df['Winter'] = df['season'].map({
        'Winter': 1,
        'Spring': 0,
        'Summer': 0,
        'Fall': 0
    })
    df['Spring'] = df['season'].map({
        'Winter': 0,
        'Spring': 1,
        'Summer': 0,
        'Fall': 0
    })
    df['Summer'] = df['season'].map({
        'Winter': 0,
        'Spring': 0,
        'Summer': 1,
        'Fall': 0
    })
    df['Fall'] = df['season'].map({
        'Winter': 0,
        'Spring': 0,
        'Summer': 0,
        'Fall': 1
    })


    df = df.drop('season', axis = 1)

    df[value_type_ids] = df[value_type_ids].interpolate(method = 'linear').bfill().ffill()

    df.info()

    train = pd.concat([train, df])

train.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 105120 entries, 2023-01-01 00:00:00 to 2023-12-31 23:55:00
Freq: 5T
Data columns (total 21 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   Unnamed: 0   105120 non-null  int64         
 1   date         105120 non-null  datetime64[ns]
 2   building_id  105120 non-null  int64         
 3   1            105120 non-null  float64       
 4   2            105120 non-null  float64       
 5   3            105120 non-null  float64       
 6   4            105120 non-null  float64       
 7   5            105120 non-null  float64       
 8   6            105120 non-null  float64       
 9   7            105120 non-null  float64       
 10  8            105120 non-null  float64       
 11  9            105120 non-null  float64       
 12  10           105120 non-null  float64       
 13  11           105120 non-null  float64       
 14  12           105120 non-null  float64    

In [ ]:
scaler = StandardScaler()   
scaler = scaler.fit(samples[value_type_ids]) #getting scaler for values 

train[value_type_ids] = scaler.transform(train[value_type_ids]) #scaling inpput values for LSTM 


In [ ]:
predictions.columns = ['device_id', 'date', 'value_type_id']
predictions = pd.merge(predictions, building_id, on='device_id', how='inner')
predictions['date'] =  pd.to_datetime(predictions['date'])
predictions['floored_date'] = predictions['date'].dt.floor('5min') #setting dates to floored 5 minutes to match time steps we used for training

### Preparing testing data

In [ ]:
building_encoder = pd.get_dummies(predictions['building_id']) 
predictions = predictions.join(building_encoder.add_suffix('_b'))

predictions['day type'] = predictions['date'].dt.dayofweek.map({
    0: 1,
    1: 1,
    2: 1,
    3: 1,
    4: 1,
    5: 0, 
    6: 0
})

predictions['work_hours'] = predictions['date'].dt.hour.between(8, 18)
predictions['work_hours'].map({True: 1, False: 0})

predictions['season'] = predictions['date'].dt.month.map({
    1: 'Winter',
    2: 'Winter',
    3: 'Spring',
    4: 'Spring',
    5: 'Spring',
    6: 'Summer',
    7: 'Summer',
    8: 'Summer',
    9: 'Fall',
    10: 'Fall',
    11: 'Fall',
    12: 'Winter'
})

season_encoder = pd.get_dummies(predictions['season'])
predictions = predictions.join(season_encoder)
predictions = predictions.drop('season', axis = 1)


In [ ]:
predictions.info() 
progressBar = tqdm() #progress bar for clarity

In [ ]:
def get_predictions(row): #get predictions for each row in test data
    type = row['value_type_id']
    date = row['floored_date'] 
    building = row['building_id']

    classes = row.drop(['device_id', 'date',  'value_type_id', 'floored_date', 'building_id']) #get classes needed for prediction

    start_date = date - timedelta(minutes=50) #get previous timesteps to feed LSTM
    inputs = train[(train['building_id'] == building)] #get timesteps in same building
    inputs = inputs.loc[(inputs['date'] >= start_date)].head(10) #10 timesteps before test date

    inputs = inputs[value_type_ids] #grab multivariate values for LSTM

    inputs = inputs.astype('float32')
    classes = classes.astype('float32')


    dataX = []
    class_X = []

    dataX.append(inputs)  # Use iloc for DataFrame slicing
    class_X.append(classes) 

    dataX = np.array(dataX) 
    class_X = np.array(class_X)


    prediction = model.predict({ #predict values
        "values": dataX,
        "class": class_X
    }, verbose = 0)


    pred = scaler.inverse_transform(prediction) #inverse scaling (because we originally scaled input data)
    
    progressBar.update(1)

    return pred[0][type - 1] #return correct prediction value





In [ ]:
progressBar = tqdm(total = len(predictions))
predictions['value'] = predictions.swifter.apply(get_predictions, axis=1) #apply predictions to all rows

In [ ]:
predictions = predictions[['device_id', 'date', 'value_type_id']]
predictions.to_csv('submission.csv', header = False)

# Next Steps and Areas for Improvement

We noticed that there were errors for constant valued value_types. For example, PM value_types seemed to take constant values. However, our model was unable to account for noise and resulted in errors. Instead, we could have improved by looking to predict PM values using the mode for that particular building_id and value_type_id. Furthermore, we did experimentation with univariate models for each value_type_id as well as models for each buildings and found a better performance. This is because the models would not find non-existant correlations between buildings and different value_type_ids. However, due to lack of time we were unable to finish predictions with them. We also did not test one hot encoding by device_id which could potentially improve performance as the expected population of a room changes the IAQ standards of the room. Finally, we could have considered better ways to handle missing values in the model creation portion. For example, filling with -1s so that our masking layer could take care of null valules instead of linearly interpolating. 